# 29-2 LLM Judge를 활용한 Evaluation

In [ ]:
# 필요한 라이브러리 임포트
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

# 평가에 활용할 모델 선언
judge_llm = init_chat_model("gpt-4o")

async def correct(outputs: dict, reference_outputs: dict) -> bool:
    # LLM에게 주는 지시사항 정의
    # 실제 답변과 기대 답변을 비교하여
    # 모든 정보가 포함되어 있는지 확인하도록 한다.
    instructions = (
        "Given an actual answer and an expected answer, determine whether"
        " the actual answer contains all of the information in the"
        " expected answer. Respond with 'CORRECT' if the actual answer"
        " does contain all of the expected information and 'INCORRECT'"
        " otherwise. Do not include anything else in your response."
    )
    
    # 그래프는 State 딕셔너리를 리턴하며
    # 'messages' 키가 있고 마지막 메시지가 AI의 최종 답변이 된다.
    actual_answer = outputs["messages"][-1].content
    expected_answer = reference_outputs["answer"]
    
    # LLM에게 전달할 메시지 구성
    # 실제 답변과 기대 답변을 명확히 구분하여 제시
    user_msg = (
        f"ACTUAL ANSWER: {actual_answer}"
        f"\n\nEXPECTED ANSWER: {expected_answer}"
    )
    
    # LLM 호출하여 답변 평가
    # system과 user 역할의 메시지를 함께 전달
    response = await judge_llm.ainvoke(
        [
            {"role": "system", "content": instructions},
            {"role": "user", "content": user_msg}
        ]
    )
    
    # LLM의 응답이 'CORRECT'인지 확인하여 결과 반환
    return response.content.upper() == "CORRECT"

In [2]:
# LangChain Hub에서 환각 평가를 위한 프롬프트 템플릿 가져오기
from langchain import hub

# RAG 시스템의 답변 환각 평가하기 위한 프롬프트 템플릿 로드
grade_prompt_hallucinations = prompt = hub.pull("langchain-ai/rag-answer-hallucination")

async def hallucination(outputs) -> dict:
    # LLM의 최종 답변과 컨텍스트 추출
    # messages 리스트의 구조:
    # - 마지막 메시지(-1): LLM의 최종 답변
    # - 마지막에서 두 번째 메시지(-2): retrieval_tool의 실행 결과로 얻은 법조문 컨텍스트
    # 이는 RAG 파이프라인에서 retrieval_tool이 먼저 실행되고, 그 결과를 바탕으로 LLM이 답변을 생성하기 때문이다.
    context = outputs["messages"][-2].content
    actual_answer = outputs["messages"][-1].content

    # LLM 응답을 위한 LCEL 활용
    # 프롬프트 템플릿과 LLM을 연결하여 평가 파이프라인 구성
    # `prompt | llm | StrOutputParser()`` 의 구조와 유사하다.
    answer_grader = grade_prompt_hallucinations | judge_llm

    # Evaluator 실행
    # 컨텍스트와 실제 답변을 입력으로 하여 환각 점수 계산
    # 환각 점수는 0~1 사이의 값으로, 1에 가까울수록 컨텍스트에 충실한 답변을 의미
    score = answer_grader.invoke({"documents": [context],
                                  "student_answer": actual_answer})
    # 점수만 추출
    score = score["Score"]

    # 평가 결과를 딕셔너리 형태로 반환
    # key: 평가 항목 이름, score: 환각 점수
    return score == 1

In [3]:
# LangSmith의 비동기 평가 함수와 소득세 에이전트 임포트
from langsmith import aevaluate
from income_tax_agent import income_tax_agent

# 입력 예제를 State 형식으로 변환하는 함수
# 사용자 질문을 messages 리스트에 추가하여 초기 상태 생성
def example_to_state(inputs: dict) -> dict:
  return {"messages": [{"role": "user", "content": inputs['question']}]}

# LCEL 선언적 구문을 사용하여 파이프라인 구성
# example_to_state의 출력을 income_tax_agent의 입력으로 연결
# langgraph 그래프는 langchain runnable이기 때문에 가능하다.
target = example_to_state | income_tax_agent

# 비동기 평가 실행
# target: 평가할 파이프라인
# data: 평가에 사용할 데이터셋 이름
# evaluators: 평가에 사용할 평가 함수 리스트 (correct 함수 사용)
experiment_results = await aevaluate(
    target,
    data="income_tax_dataset",
    evaluators=[correct, hallucination],
    experiment_prefix='gpt-4o',
    description='1.0.1 운영 배포 전 최종 점검'
)

/Users/jasonkang/personal/langgraph-book/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'gpt-4o-470343d4' at:
https://smith.langchain.com/o/0c1355d3-1674-4800-b34c-a6eadd4860d5/datasets/ce85bc47-501b-41d3-be3a-9ab56f1524bd/compare?selectedSessions=1c09caa6-d5fe-4371-93f3-17f1bd3b8159




0it [00:00, ?it/s]Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10f6837a0>
1it [00:17, 17.22s/it]Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10ce1a960>
3it [00:36, 11.60s/it]Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1291b7920>
4it [00:48, 11.50s/it]Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10fcf8ad0>
5it [00:57, 10.54s/it]Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10fe0c110>
7it [01:16,  9.88s/it]Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10fe521b0>
8it [01:26,  9.98s/it]Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10fe50ec0>
9it [01:37, 10.49s/it]Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10fe89ca0>
10it [01:46,  9.75s/it]Unclosed client session
client_session: <aiohttp.client.Clien